# Assignment 10

### 1.复习上课内容

### 2. 回答一下理论题目

#### 1. What is independent assumption in Naive bayes ?

事件之间相互独立，没有关联联系。

#### 2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?

MAP: $h_{MAP}= \mathop{\arg\max}_{h\in H} p(h|D) = \mathop{\arg\max}_{h\in H}\frac{p(D|h)·p(h)}{p(D)} = \mathop{\arg\max}_{h\in H} p(D|h)·p(h) $  

若h分布均匀，可用ML: $h_{ML}=\mathop{\arg\max}_{h\in H} p(D|h)$

#### 3. What is support vector in SVM?

support vector为支持向量 -> 对边界确定有贡献的向量 -> $\alpha_{i}>0$

#### 4. What is the intuition behind SVM ?

一个多维平面分布着正样本和负样本，如何找到一个超平面将正负样本分开。  
- 超平面存在,能够将正负样本分开 -> 线性可分SVM
- 超平面不存在 -> 正负样本彻底混乱不可分(核函数)，少数样本不可分(松弛变量)

#### 5. Shortly describe what 'random' means in random forest ?

- 随机选取部分训练样本
- 随机选取部分特征构建树

#### 6. What cariterion does XGBoost use to find the best split point in a tree ?

选择某个分割点 -> 计算该分割点下最优的权重$w_j=-\frac{G_j}{Hj+\lambda}$ -> 计算$obj=-\frac{1}{2}\sum_{j=1}^{T} \frac{G_j^2}{H_j+\lambda}+\gamma T$ -> 最小的作为最佳分割点 -> 如果obj>0,则该树停止分裂 -> 建下一棵树

### 3. Practial part

##### Problem description: In this part you are going to build a classifier to detect if a piece of news is published by the Xinhua news agency (新华社）.

#### Hints:

###### 1. Firstly, you have to come up with a way to represent the news. (Vectorize the sentence, you can find different ways to do so online)  

Reference:  
https://blog.csdn.net/miner_zhu/article/details/82844922  
https://blog.csdn.net/XnCSD/article/details/86742377  
https://www.cnblogs.com/jiangxinyang/category/1374701.html 文本分类实战  
https://github.com/wavewangyue/text-classification 机器学习大乱斗 2017.05.05  

**[1]Preprocessing**

In [32]:
import pandas as pd
filename = r'.\data\sqlResult_1558435.csv'
content = pd.read_csv(filename,encoding='gb18030')
article = content['content'].tolist() #共有89611篇,新华社78661，非新华社10950篇

In [4]:
# 清洁数据
import re
def token(string):
    return re.findall('\w+',string) 
def article_clean(article_origin):
    article_cleaned=[''.join(token(str(a))) for a in article_origin]
    return article_cleaned
article_cleaned = article_clean(article)
'''
with open('.\data\cleaned.txt','w',encoding="utf-8") as f:
    for a in article_cleaned:
        f.write(a+'\n')
'''

'\nwith open(\'.\\cleaned.txt\',\'w\',encoding="utf-8") as f:\n    for a in article_cleaned:\n        f.write(a+\'\n\')\n'

In [41]:
# 分词
import jieba
def cut(string):
    cut_string = [' '.join(jieba.cut(string))]
    return cut_string
def cut_text(origin_file,cuted_file):
    TOKEN = []
    for i,line in enumerate((open(origin_file,encoding="utf-8"))):
        if i%10000==0:
            print(i)
        if line != "":
            TOKEN += cut(line)
    return TOKEN
TOKEN = cut_text('.\data\cleaned.txt','.\data\cuted.txt')

with open('.\data\cuted.txt','w',encoding="utf-8") as f:
    for a in TOKEN:
        f.write(a)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wy\AppData\Local\Temp\jieba.cache


0


Loading model cost 0.961 seconds.
Prefix dict has been built successfully.


10000
20000
30000
40000
50000
60000
70000
80000


In [76]:
# 训练word2vec模型
import gensim
import logging
from gensim.models import word2vec
from gensim.models import Word2Vec

#设置输出日志
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#直接用gemsim提供的API去读取txt文件，读取文件的API有LineSentence 和 Text8Corpus, PathLineSentences等
sentences = word2vec.LineSentence('.\data\cuted.txt')
#训练模型，词向量长度设置为100，采用skip-gram模型，模型保存为bin格式
model = Word2Vec(sentences,size=100,sg=1)
model.wv.save_word2vec_format(".\data\word2Vec"+".bin",binary=True) #获取词向量并存为二进制文件形式
#从文件中获取词向量
wordVec = gensim.models.KeyedVectors.load_word2vec_format(".\data\word2Vec.bin",binary=True)

2020-03-29 10:28:57,403 : INFO : collecting all words and their counts
2020-03-29 10:28:57,405 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-03-29 10:28:58,753 : INFO : PROGRESS: at sentence #10000, processed 3684164 words, keeping 148557 word types
2020-03-29 10:28:59,379 : INFO : PROGRESS: at sentence #20000, processed 5353913 words, keeping 174173 word types
2020-03-29 10:28:59,961 : INFO : PROGRESS: at sentence #30000, processed 6948716 words, keeping 189746 word types
2020-03-29 10:29:00,552 : INFO : PROGRESS: at sentence #40000, processed 8626790 words, keeping 206181 word types
2020-03-29 10:29:01,191 : INFO : PROGRESS: at sentence #50000, processed 10484049 words, keeping 222534 word types
2020-03-29 10:29:01,856 : INFO : PROGRESS: at sentence #60000, processed 12378660 words, keeping 237871 word types
2020-03-29 10:29:02,440 : INFO : PROGRESS: at sentence #70000, processed 14084045 words, keeping 250282 word types
2020-03-29 10:29:03,033 : INF

2020-03-29 10:30:22,564 : INFO : EPOCH 1 - PROGRESS: at 80.36% examples, 221482 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:30:23,592 : INFO : EPOCH 1 - PROGRESS: at 81.92% examples, 221104 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:30:24,647 : INFO : EPOCH 1 - PROGRESS: at 83.45% examples, 221068 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:30:25,683 : INFO : EPOCH 1 - PROGRESS: at 85.23% examples, 221091 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:30:26,702 : INFO : EPOCH 1 - PROGRESS: at 86.94% examples, 221169 words/s, in_qsize 6, out_qsize 0
2020-03-29 10:30:27,739 : INFO : EPOCH 1 - PROGRESS: at 88.90% examples, 221168 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:30:28,776 : INFO : EPOCH 1 - PROGRESS: at 90.70% examples, 221044 words/s, in_qsize 6, out_qsize 0
2020-03-29 10:30:29,801 : INFO : EPOCH 1 - PROGRESS: at 92.07% examples, 220563 words/s, in_qsize 6, out_qsize 0
2020-03-29 10:30:30,826 : INFO : EPOCH 1 - PROGRESS: at 93.18% examples, 220090 words/s, in_qsiz

2020-03-29 10:31:34,778 : INFO : EPOCH 2 - PROGRESS: at 81.01% examples, 217201 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:31:35,804 : INFO : EPOCH 2 - PROGRESS: at 82.79% examples, 217339 words/s, in_qsize 6, out_qsize 0
2020-03-29 10:31:36,859 : INFO : EPOCH 2 - PROGRESS: at 84.42% examples, 217362 words/s, in_qsize 4, out_qsize 1
2020-03-29 10:31:37,900 : INFO : EPOCH 2 - PROGRESS: at 86.17% examples, 217423 words/s, in_qsize 6, out_qsize 0
2020-03-29 10:31:38,930 : INFO : EPOCH 2 - PROGRESS: at 87.97% examples, 217504 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:31:39,957 : INFO : EPOCH 2 - PROGRESS: at 89.92% examples, 217603 words/s, in_qsize 6, out_qsize 0
2020-03-29 10:31:40,958 : INFO : EPOCH 2 - PROGRESS: at 91.42% examples, 217651 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:31:41,996 : INFO : EPOCH 2 - PROGRESS: at 92.89% examples, 217570 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:31:43,065 : INFO : EPOCH 2 - PROGRESS: at 94.57% examples, 217531 words/s, in_qsiz

2020-03-29 10:32:46,208 : INFO : EPOCH 3 - PROGRESS: at 82.28% examples, 217616 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:32:47,292 : INFO : EPOCH 3 - PROGRESS: at 83.90% examples, 217540 words/s, in_qsize 4, out_qsize 1
2020-03-29 10:32:48,336 : INFO : EPOCH 3 - PROGRESS: at 85.62% examples, 217574 words/s, in_qsize 6, out_qsize 0
2020-03-29 10:32:49,386 : INFO : EPOCH 3 - PROGRESS: at 87.45% examples, 217592 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:32:50,407 : INFO : EPOCH 3 - PROGRESS: at 89.44% examples, 217717 words/s, in_qsize 6, out_qsize 0
2020-03-29 10:32:51,457 : INFO : EPOCH 3 - PROGRESS: at 91.04% examples, 217736 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:32:52,488 : INFO : EPOCH 3 - PROGRESS: at 92.53% examples, 217671 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:32:53,505 : INFO : EPOCH 3 - PROGRESS: at 94.13% examples, 217801 words/s, in_qsize 6, out_qsize 0
2020-03-29 10:32:54,512 : INFO : EPOCH 3 - PROGRESS: at 95.93% examples, 217855 words/s, in_qsiz

2020-03-29 10:33:58,292 : INFO : EPOCH 4 - PROGRESS: at 81.36% examples, 210951 words/s, in_qsize 6, out_qsize 0
2020-03-29 10:33:59,326 : INFO : EPOCH 4 - PROGRESS: at 83.01% examples, 211022 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:34:00,336 : INFO : EPOCH 4 - PROGRESS: at 84.63% examples, 211168 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:34:01,354 : INFO : EPOCH 4 - PROGRESS: at 86.21% examples, 211272 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:34:02,355 : INFO : EPOCH 4 - PROGRESS: at 87.97% examples, 211410 words/s, in_qsize 6, out_qsize 0
2020-03-29 10:34:03,360 : INFO : EPOCH 4 - PROGRESS: at 89.89% examples, 211542 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:34:04,397 : INFO : EPOCH 4 - PROGRESS: at 91.32% examples, 211564 words/s, in_qsize 6, out_qsize 0
2020-03-29 10:34:05,449 : INFO : EPOCH 4 - PROGRESS: at 92.89% examples, 211659 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:34:06,495 : INFO : EPOCH 4 - PROGRESS: at 94.57% examples, 211789 words/s, in_qsiz

2020-03-29 10:35:09,910 : INFO : EPOCH 5 - PROGRESS: at 81.49% examples, 214942 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:35:10,915 : INFO : EPOCH 5 - PROGRESS: at 83.03% examples, 214917 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:35:11,940 : INFO : EPOCH 5 - PROGRESS: at 84.69% examples, 214948 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:35:12,956 : INFO : EPOCH 5 - PROGRESS: at 86.26% examples, 214987 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:35:13,995 : INFO : EPOCH 5 - PROGRESS: at 88.11% examples, 214948 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:35:14,996 : INFO : EPOCH 5 - PROGRESS: at 89.92% examples, 215046 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:35:16,019 : INFO : EPOCH 5 - PROGRESS: at 91.42% examples, 215055 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:35:17,056 : INFO : EPOCH 5 - PROGRESS: at 92.89% examples, 215022 words/s, in_qsize 5, out_qsize 0
2020-03-29 10:35:18,127 : INFO : EPOCH 5 - PROGRESS: at 94.57% examples, 215017 words/s, in_qsiz

In [ ]:
#构建数据集
import pandas as pd
contents = []
with open('.\cuted.txt','r',encoding="utf-8") as f:
    a = f.readlines()
    contents += a

filename = r'.\data\sqlResult_1558435.csv' #...:the path of the file
content = pd.read_csv(filename,encoding='gb18030')
target = content['source'].tolist()
target = [1 if i=="新华社" else 0 for i in target]

data = []
data.append(contents)
data.append(target)
np_data = np.array(data).T
save = pd.DataFrame(np_data,columns=['content','target'])
#save.to_csv('.\dataset.csv',index=False,header=True)

**[2] Data Processing** 

In [ ]:
import csv
import pandas as pd
import numpy as np
import json
import gensim
from collections import Counter
from sklearn.model_selection import train_test_split

In [39]:
#sequenceLength取平均值
content = pd.read_csv('.\data\dataset.csv',encoding='utf-8')
news = [line.strip().split() for line in content['content']]
sum=0
for i in range(len(news)):
    sum += len(news[i])
meanLength = sum/len(news)  #meanLength=195,因此我们取sequenceLength=200

195.60816194440415


In [ ]:
class Config(object):
    sequenceLength = 200
    dataSoource = '.\data\dataset.csv'
    word2cevSource = '.\data\word2Vec.bin'
    rate = 0.8

In [ ]:
config = Config()

In [ ]:
class Dataset(object):
    def __init__(self, config):
        self.config = config
        self._dataSource = config.dataSource
        self._word2vecSource = config.word2vecSource
        self._rate = config.rate
        self._sequenceLength = config.sequenceLength
        self._embeddingSize = 100  # 词向量的大小

        self.trainNews = []
        self.trainLabels = []
        self.testNews = []
        self.testLabels = []
        self.wordEmbedding = None
        self.labelList = []

    def _readData(self, dataPath):
        """
        从csv文件中读取数据集
        """
        content = pd.read_csv(dataPath, encoding='utf-8')
        news = [line.strip().split() for line in content['content']]
        labels = content['target']
        return news, labels

    def _getWordEmbedding(self, words):
        """
        根据所给语料，取出预训练好的word2vec中存在的词向量
        """
        wordVec = gensim.models.KeyedVectors.load_word2vec_format(self._word2vecSource, binary=True)
        vocab = []
        wordEmbedding = []
        # 添加"PAD"和"UNK","PAD"用于补充序列长度，"UNK"用于代替词典中没有的词
        vocab.append("PAD")
        vocab.append("UNK")
        wordEmbedding.append(np.zeros(self._embeddingSize))
        wordEmbedding.append(np.random.randn(self._embeddingSize))

        for word in words:
            try:
                vector = wordVec.wv[word]
                vocab.append(word)
                wordEmbedding.append(vector)
            except:
                print(word + "不存在于词向量中")
        return vocab, np.array(wordEmbedding)

    def _genVocabulary(self, news):
        """
        生成词向量-索引和词汇-索引映射字典
        若既有有标签数据集和无标签数据集，可以用全数据集(当同时有有标签数据集和无标签数据集时)
        复杂情况下，可在该步添加去除停用词或低频词等操作，本次不添加这些操作
        """
        words = [word for new in news for word in new]
        wordCount = Counter(words)
        sortWordCount = sorted(wordCount.items(), key=lambda x: x[1], reverse=True)
        words = [item[0] for item in sortWordCount if item[1] >= 1]  # 该步可用于去除低频次
        vocab, wordEmbedding = self._getWordEmbedding(words)
        self.wordEmbedding = wordEmbedding

        word2idx = dict(zip(vocab, list(range(len(vocab)))))
        label2idx = dict(zip(["非新华社","新华社"],list(range(2))))
        self.labelList = list(range(2))

        # 将词汇-索引映射表保存为json数据，之后做inference时直接加载来处理数据
        with open('.\data\word2idx.json', 'w', encoding="utf-8") as f:
            json.dump(word2idx, f)
        with open('.\data\label2idx.json', 'w', encoding="utf-8") as f:
            json.dump(label2idx, f)
        return word2idx, label2idx

    def _wordToIndex(self, news, word2idx):
        """
        将词转换成索引
        """
        newsIds = [[word2idx.get(item, word2idx["UNK"]) for item in new] for new in news]
        return newsIds

    def _genTrainEvalData(self, x, y, word2idx):
        """
        生成训练集和测试集
        x - 经过转换成wordIndex的数据集
        y - 标签
        """
        # 设定固定序列长度，多截少补
        news = []
        for new in x:
            if len(new) >= self._sequenceLength:
                news.append(new[:self._sequenceLength])
            else:
                news.append(new + [word2idx["PAD"]] * (self._sequenceLength - len(new)))
        labels = y
        x_train,x_test,y_train,y_test = train_test_split(news,labels,test_size=1-self._rate,random_state=1)
        #assert len(news) == len(x_train) + len(x_test)
        #assert len(labels) == len(x_test) + len(y_test)
        return x_train, y_train, x_test, y_test

    def dataGen(self):
        """
        初始化训练集和测试集
        """
        # 初始化数据集
        news, labels = self._readData(self._dataSource)
        # 初始化词汇-索引映射表和词向量矩阵
        word2idx, label2idx = self._genVocabulary(news)
        # 将标签和句子数值化
        newsIds = self._wordToIndex(news, word2idx)
        labelIds = labels
        # 初始化训练集和测试集
        x_train, y_train, x_test, y_test = self._genTrainEvalData(newsIds,labels,word2idx)
        self.trainNews = x_train
        self.trainLabels = y_train
        self.testNews = x_test
        self.testLabels = y_test

In [ ]:
data = Dataset(config)
data.dataGen()

**[3] Build Model**

###### 2. Secondly,  pick a machine learning algorithm that you think is suitable for this task

### Congratulations! You have completed all assignments in this week. The question below is optional. If you still have time, why don't try it out.

## Option:

#### Try differnt machine learning algorithms with different combinations of parameters in the practical part, and compare their performances (Better use some visualization techiniques).